In [1]:
import numpy as np
import pandas as pd
import os
import datetime as dt
print(os.listdir("/home/sam2220/Desktop/ML_projects/NY_taxi_fare_prediction/kaggle_challenge_dataset"))

['test.csv', '.~lock.sample_submission.csv#', 'sample_submission.csv', 'train.csv', 'GCP-Coupons-Instructions.rtf']


In [2]:
#read train and test dataset 
TRAIN_PATH = "/home/sam2220/Desktop/ML_projects/NY_taxi_fare_prediction/kaggle_challenge_dataset/train.csv"
TEST_PATH = "/home/sam2220/Desktop/ML_projects/NY_taxi_fare_prediction/kaggle_challenge_dataset/test.csv"
with open(TRAIN_PATH) as file:
    len_train = len(file.readlines())

print(len_train)

55423857


In [3]:
train_temp = pd.read_csv(TRAIN_PATH, nrows=5)
print("training dataset head:")
print(train_temp)
#test_temp = pd.read_csv(TEST_PATH, nrows=5)
#print("test dataset head:")
#print(test_temp)

training dataset head:
                             key  fare_amount          pickup_datetime  \
0    2009-06-15 17:26:21.0000001          4.5  2009-06-15 17:26:21 UTC   
1    2010-01-05 16:52:16.0000002         16.9  2010-01-05 16:52:16 UTC   
2   2011-08-18 00:35:00.00000049          5.7  2011-08-18 00:35:00 UTC   
3    2012-04-21 04:30:42.0000001          7.7  2012-04-21 04:30:42 UTC   
4  2010-03-09 07:51:00.000000135          5.3  2010-03-09 07:51:00 UTC   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.844311        40.721319         -73.841610         40.712278   
1        -74.016048        40.711303         -73.979268         40.782004   
2        -73.982738        40.761270         -73.991242         40.750562   
3        -73.987130        40.733143         -73.991567         40.758092   
4        -73.968095        40.768008         -73.956655         40.783762   

   passenger_count  
0                1  
1                1  
2     

In [8]:
#chunksize = 1000

train_types = {'fare_amount': 'float32',
              'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

test_types =  {'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

train_cols = list(train_types.keys())
test_cols = list(test_types.keys())

def clean_df(PATH, cols, datatypes, chunksize):
    if PATH == TEST_PATH:
        df_list = []
        for chunk in pd.read_csv(PATH, usecols = cols, dtype=datatypes, chunksize=chunksize):
            chunk['pickup_datetime'] = chunk['pickup_datetime'].str.slice(0, 16)
            chunk['pickup_datetime'] = pd.to_datetime(chunk['pickup_datetime'], utc=True, 
                                                      format='%Y-%m-%d %H:%M')

            df_list.append(chunk)
            #print(len(df_list))
        return pd.concat(df_list)
    
    elif(PATH == TRAIN_PATH):
        X_df_list = []
        y_df_list = []
        #df_list = []
        for chunk in pd.read_csv(PATH, usecols = cols, dtype=datatypes, chunksize=chunksize):  
            #converting pickup datetime to proper format
            chunk['pickup_datetime'] = chunk['pickup_datetime'].str.slice(0, 16)
            chunk['pickup_datetime'] = pd.to_datetime(chunk['pickup_datetime'], utc=True, 
                                                      format='%Y-%m-%d %H:%M')
            #outlier removal
            #remove rows with passenger count 0 and more than 6
            chunk = chunk[(chunk['passenger_count'] > 0) & (chunk['passenger_count'] < 7) ]
            
            #remove rows with fare amount less than $2 and greater than $400 
            chunk = chunk[(chunk['fare_amount'] > 2.0) & (chunk['fare_amount'] < 400.0)]            
            
            #add datetime features to the data 
            #pickup_datetime -> hour, month, year, isWeekday, day
            add_datetime_features(chunk)
        
            #appending chunks to the dataframe list
            #df_list.append(chunk)
            y_df_list.append(chunk['fare_amount'])
            chunk.drop(labels = 'fare_amount', axis=1, inplace=True)
            X_df_list.append(chunk)
            
        
        #df = pd.concat(df_list)
        X_df = pd.concat(X_df_list)
        y_df = pd.concat(y_df_list)
        return  X_df, y_df

def add_datetime_features(data_chunk):
    data_chunk['hour'] = data_chunk.pickup_datetime.dt.hour
    data_chunk['day'] = data_chunk.pickup_datetime.dt.day
    data_chunk['month'] = data_chunk.pickup_datetime.dt.month
    data_chunk['weekday'] = data_chunk.pickup_datetime.dt.weekday
    data_chunk['year'] = data_chunk.pickup_datetime.dt.year
    
    return data_chunk
    
#insample dataset
train_target, train_label = clean_df(TRAIN_PATH, train_cols, train_types, chunksize = 5000000)
#n_train = len(train_label)
print(train_target.shape)

#out of samples dataset
test_dataset = clean_df(TEST_PATH, test_cols, test_types, chunksize = 2000)
n_test = len(test_dataset)
print(n_test)

(55223448, 11)
9914


In [9]:
train_target.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day,month,weekday,year
count,5.522345e+07,5.522345e+07,5.522345e+07,5.522345e+07,5.522345e+07,5.522345e+07,5.522345e+07,5.522345e+07,5.522345e+07,5.522345e+07
mean,-3.889352e+01,1.944359e+01,-3.889297e+01,1.944358e+01,1.691095e+00,1.350937e+01,1.571500e+01,6.268004e+00,3.041398e+00,2.011740e+03
std,2.560817e+01,1.416243e+01,2.558437e+01,1.414724e+01,1.307215e+00,6.518191e+00,8.684342e+00,3.435156e+00,1.948963e+00,1.862780e+00
min,-3.442060e+03,-3.492264e+03,-3.442025e+03,-3.547887e+03,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.009000e+03
25%,-7.399207e+01,4.073493e+01,-7.399140e+01,4.073403e+01,1.000000e+00,9.000000e+00,8.000000e+00,3.000000e+00,1.000000e+00,2.010000e+03
50%,-7.398180e+01,4.075265e+01,-7.398015e+01,4.075315e+01,1.000000e+00,1.400000e+01,1.600000e+01,6.000000e+00,3.000000e+00,2.012000e+03
75%,-7.396708e+01,4.076713e+01,-7.396367e+01,4.076810e+01,2.000000e+00,1.900000e+01,2.300000e+01,9.000000e+00,5.000000e+00,2.013000e+03
max,3.457626e+03,3.408790e+03,3.457622e+03,3.537133e+03,6.000000e+00,2.300000e+01,3.100000e+01,1.200000e+01,6.000000e+00,2.015000e+03
